In [5]:
import numpy as np
from astropy.coordinates import get_sun
import astropy, h5py, os
import astropy.time as Time
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt
from scipy.misc import logsumexp
from gwpy.timeseries import TimeSeries
import lal, gwpy
%matplotlib inline

ImportError: No module named antres

In [1]:
# Data filtering 

def iir_bandstops(fstops, fs, order=4):
    import numpy as np
    from scipy.signal import iirdesign, zpk2tf, freqz
    nyq = 0.5 * fs
    # Zeros zd, poles pd, and gain kd for the digital filter
    zd = np.array([])
    pd = np.array([])
    kd = 1

    # Notches
    for fstopData in fstops:
        fstop = fstopData[0]
        df = fstopData[1]
        df2 = fstopData[2]
        low = (fstop - df) / nyq
        high = (fstop + df) / nyq
        low2 = (fstop - df2) / nyq
        high2 = (fstop + df2) / nyq
        z, p, k = iirdesign([low,high], [low2,high2], gpass=1, gstop=6,
                            ftype='ellip', output='zpk')
        zd = np.append(zd,z)
        pd = np.append(pd,p)

    # Set gain to one at 100 Hz...better not notch there
    bPrelim,aPrelim = zpk2tf(zd, pd, 1)
    outFreq, outg0 = freqz(bPrelim, aPrelim, 100/nyq)

    # Return the numerator and denominator of the digital filter
    b,a = zpk2tf(zd,pd,k)
    return b, a

def get_filter_coefs(det,fs=4096):
    from scipy.signal import butter, filtfilt, iirdesign, zpk2tf, freqz
    import numpy as np
    from notchfilt import iir_bandstops

    # assemble the filter b,a coefficients:
    coefs = []

    # bandpass filter parameters
    lowcut = 100
    highcut = 150
    order = 4

    # bandpass filter coefficients
    nyq = 0.5*fs
    low = lowcut / nyq
    high = highcut / nyq
    bb, ab = butter(order, [low, high], btype='band')
    coefs.append((bb,ab))

    # Frequencies of notches at known instrumental spectral line frequencies.
    # You can see these lines in the ASD above, so it is straightforward to make this list.
    if det=='L1':
        notchesAbsolute = np.array([120.0, 139.94, 145.06, 108.992])
    elif det=='H1':
        notchesAbsolute = np.array([120.0, 139.95, 140.41, 108.992])
    else:
        print 'Error: Detector can only be H1 or L1'
        exit()
    # notch filter coefficients:
    for notchf in notchesAbsolute:
        bn, an = iir_bandstops(np.array([[notchf,1,0.1]]), fs, order=4)
        coefs.append((bn,an))

    return coefs

# and then define the filter function:
def filter_data(data_in,coefs):
    from scipy.signal import filtfilt
    data = data_in.copy()
    for coef in coefs:
        b,a = coef
        # filtfilt applies a linear filter twice, once forward and once backwards.
        # The combined filter has linear phase.
        data = filtfilt(b, a, data)
    return data


In [4]:
# Antenna Respone 
def antenna_response( gpsTime, ra, dec, psi, det ):
    from types import StringType, FloatType
    import lal
    import numpy as np
    from types import StringType, FloatType

    # create detector-name map
    detMap = {'H1': lal.LALDetectorIndexLHODIFF, \
            'H2': lal.LALDetectorIndexLHODIFF, \
            'L1': lal.LALDetectorIndexLLODIFF, \
            'G1': lal.LALDetectorIndexGEO600DIFF, \
            'V1': lal.LALDetectorIndexVIRGODIFF, \
            'T1': lal.LALDetectorIndexTAMA300DIFF, \
            'AL1': lal.LALDetectorIndexLLODIFF, \
            'AH1': lal.LALDetectorIndexLHODIFF, \
            'AV1': lal.LALDetectorIndexVIRGODIFF}

    try:
        detector=detMap[det]
    except KeyError:
        raise ValueError, "ERROR. Key %s is not a valid detector name." % (det)

    # get detector
    detval = lal.CachedDetectors[detector]
    response = detval.response

    # check if gpsTime is just a float or int, and if so convert into an array
    if isinstance(gpsTime, float) or isinstance(gpsTime, int):
        gpsTime = np.array([gpsTime])
    else: # make sure it's a numpy array
        gpsTime = np.copy(gpsTime)

    # if gpsTime is a list of regularly spaced values then use ComputeDetAMResponseSeries
    if len(gpsTime) == 1 or np.unique(np.diff(gpsTime)).size == 1:
        gpsStart = lal.LIGOTimeGPS( gpsTime[0] )
        dt = 0.
    if len(gpsTime) > 1:
        dt = gpsTime[1]-gpsTime[0]
        fp, fc = lal.ComputeDetAMResponseSeries(response, ra, dec, psi, gpsStart, dt, len(gpsTime))

    # return elements from Time Series
        return fp.data.data, fc.data.data
    
    else: # we'll have to work out the value at each point in the time series
        fp = np.zeros(len(gpsTime))
        fc = np.zeros(len(gpsTime))

    for i in range(len(gpsTime)):
        gps = lal.LIGOTimeGPS( gpsTime[i] )
        gmst_rad = lal.GreenwichMeanSiderealTime(gps)

        # actual computation of antenna factors
        fp[i], fc[i] = lal.ComputeDetAMResponse(response, ra, dec, psi, gmst_rad)
    fp = fp[0]
    fc = fc[0]
    return fp, fc



In [ ]:
# Analysis
# code to add up the probability for a month or a week, prints the duration, and plots the cumulative probability to infer the upper limit at x% confidence. Prints the upper limit on h0 at 95% confidence
def plotdist(starttime,endtime):
	# starttime and endtime should be in format 'week'+str(i) or 'month'+str(i) only.
	# In the near future it will take 'all' as starttime and endtime to plot the entire data.
	import numpy as np
	from matplotlib.backends.backend_pdf import PdfPages
	import matplotlib.pyplot as plt
	import os
	fname =	'probdist_cumulative.pdf'
	if os.path.exists('/home/spxha/')==True:
		pathtointersect = '/home/spxha/solarGW/intersect_old.txt'
	else:
		pathtointersect = '../../../intersect_old.txt'
	timearray = np.array(np.loadtxt(pathtointersect,dtype='f8'))
	StartTimes = timearray[:,0]
	EndTimes   = timearray[:,1]
	if starttime=='all' and endtime=='all':
		starttime = 931076896.0
		endtime   = 971614889.0
	duration = 0
	newStartTimes = StartTimes[(np.abs(StartTimes-starttime)).argmin():(np.abs(EndTimes-endtime)).argmin()]
	newEndTimes   =   EndTimes[(np.abs(StartTimes-starttime)).argmin():(np.abs(EndTimes-endtime)).argmin()]
	p_array,h0_array = [[[0 for _ in range(30)] for _ in range(len(newStartTimes))] for _ in range(2)]
	j = 0
	for i in range(len(newStartTimes)):
		pathi = 'p'+str(int(newStartTimes[i]))+'.txt'
		if os.path.exists(pathi)==True:
			j +=1
			print i
			p_array[i]  = np.array(np.loadtxt('p'+str(int(newStartTimes[i]))+'.txt',dtype='float'))
			h0_array[i] = np.array(np.loadtxt('h0'+str(int(newStartTimes[i]))+'.txt',dtype='float'))
			duration += newEndTimes[i]-newStartTimes[i] - 150
		else:
			pass
	print j
	p_sum_array = [0.0 for _ in range(30)]
	p_sum_array = np.array(p_sum_array)
	h0_array = np.array(h0_array)
	print p_array[1]
	p_array = np.array(p_array)
	for i in range(len(p_array)-1):
		p_sum_array += p_array[i+1]
	h0_mean_array = h0_array.mean(0)
	p = np.exp(p_sum_array-np.max(p_sum_array))
	print h0_mean_array,p

	# Making a cumulative plot to get the 95% value.
	prob = [0 for _ in range(len(p))]
	for i in range(30):
		d = [0 for _ in range(i+1)]
		print i
		for j in range(i+1):
			print i,j
			d[j] = p[i-j]
			print d[j]
		prob[i] = np.sum(d)
		print prob[i]
	prob = prob/(np.max(prob))
	print prob
	# plot the distribution
	with PdfPages(fname) as pdf:
		fig1 = plt.figure()
		plt.plot(h0_mean_array,prob,'+')
		plt.title('Cumulative probability for '+str(int(duration/3600))+' hours')
		plt.xlabel(r'$h_0$')
		plt.ylabel('Normalised cumulative probability')
		pdf.savefig(fig1)
		plt.close()
	print (h0_mean_array[10]+h0_mean_array[11])/2
